In [31]:
def isNebor(edges,point1,point2):
    if point1 not in edges: return False
    ind=edges.index(point1)
    if point2 in [edges[ind-1],edges[(ind+1)%len(edges)]]:
        return True
    return False

In [32]:
def getCrossSections(edges,xyRanges):
    xMin,yMin=xyRanges[0]
    xMax,yMax=xyRanges[1]
    css=[]
    for x in range(xMin,xMax+1):
        cs=[]
        for y in range(yMin,yMax):
            if isNebor(edges,(x,y),(x,y+1)):
                cs.append(1)
            else:
                cs.append(0)
        css.append(cs)
    return css

In [36]:
def xorSegments(window,segments):
    for i in range(len(window)):
        window[i]=window[i]^segments[i]

In [49]:
def getArea(edges,xyRanges):
    xMin,yMin=xyRanges[0]
    xMax,yMax=xyRanges[1]
    window=[0 for _ in range(yMin,yMax)]
    css=getCrossSections(edges,xyRanges)
    area=0
    for cs in css:
        xorSegments(window,cs)
        area+=sum(window)
    return area 

In [59]:
getArea(edges,xyRanges)

2190

## <font face='comic sans ms' color ='yellow'>Old</font>

In [1]:
def getDistance(steps):
    return steps.count('F')

In [2]:
def getSteps(data):
    steps=''
    for i in range(1,len(data),2):
        steps+=data[i]*int(data[i+1])
    return steps

In [3]:
def getMaxArea(xyRanges):
    min_x=xyRanges[0][0]
    min_y=xyRanges[0][1]
    max_x=xyRanges[1][0]
    max_y=xyRanges[1][1]
    return (max_x-min_x)*(max_y-min_y)

In [4]:
def getXYranges(edges):
    xyRanges=[]
    x_points=[x for x,_ in edges]
    y_points=[y for _,y in edges]
    max_x=max(x_points)
    min_x=min(x_points)
    max_y=max(y_points)
    min_y=min(y_points)
    xyRanges.append((min_x,min_y))
    xyRanges.append((max_x,max_y))
    return xyRanges

In [5]:
def getEdges(steps):
    direc=0
    northSteps=[0]
    eastSteps=[0]
    edgePoints=[]
    for step in steps:
        if step in 'RL':
            direc+=1 if step=='R' else -1
        else:
            edgePoints.append((eastSteps[-1],northSteps[-1]))
            if direc%2==0:
                di=1 if direc%4==0 else -1
                northSteps.append(northSteps[-1]+di)
            else:
                di=1 if (direc-1)%4==0 else -1
                eastSteps.append(eastSteps[-1]+di)
    return edgePoints

In [6]:
def getCrossSection(edges,xyRanges):
    xMin,yMin=xyRanges[0]
    xMax,yMax=xyRanges[1]
    csa=[]
    for x in range(xMin,xMax+1):
        aCsa=[]
        for y in range(yMin,yMax+1):
            if (x,y) in edges:
                aCsa.append((x,y))
        csa.append(aCsa)
    return csa 

In [7]:
def getSegments(edges,csa):
    segments=[]
    for i in range(len(csa)-1):
        x=edges.index(csa[i])
        if csa[i+1] in [edges[x-1],edges[(x+1)%len(edges)]]:
            segments.append([csa[i],csa[i+1]])
    return segments

In [8]:
def xorSegments(window,segments):
    for segment in segments:
        if segment in window:
            window.remove(segment)
        elif ([segment[1],segment[0]]) in window:
            window.remove([segment[1],segment[0]])
        else:
            window.append(segment)

In [9]:
def moveWindow(window):
    for i in range(len(window)):
        [(x1,y1),(x2,y2)]=window[i]
        window[i]=[(x1+1,y1),(x2+1,y2)]

In [85]:
def getArea(edges,xyRanges):
    csa=getCrossSection(edges,xyRanges)
    window=[]
    antiWin=[[(xyRanges[0][0],i),(xyRanges[0][0],i+1)] for i in range(xyRanges[0][1],xyRanges[1][1])]
    area=0
    pocketArea=0
    # print(antiWin)
    for cs in csa:
        xorSegments(window,getSegments(edges,cs))
        xorSegments(antiWin,getSegments(edges,cs))
        pocketArea+=getPoketArea(edges,antiWin,xyRanges)
        # print(antiWin)
        moveWindow(antiWin)
        moveWindow(window)
        area+=len(window)
    return area,pocketArea

In [86]:
def getPoketArea(edges,antiWin,xyRanges):
    c=0
    for segment in antiWin:
        if isSegmentInPocket(edges,segment,xyRanges):
            c+=1
    return c

In [102]:
trails=input()
data=trails.split()
steps=getSteps(data)
edges=getEdges(steps)

In [103]:
xyRanges=getXYranges(edges)
xyRanges

[(-18, -55), (30, 19)]

In [104]:
getDistance(steps)

344

In [105]:
getMaxArea(xyRanges)

3552

In [106]:
getArea(edges,xyRanges)

(2190, 111)

In [83]:
def isSegmentInPocket(edges,segment,xyRanges):
    xMin,yMin=xyRanges[0]
    xMax,yMax=xyRanges[1]
    [(x1,y1),(x2,y2)]=segment
    # checking North and South (up & down)
    c=0
    for i in range(y1,yMin-1,-1):
        if (x1,i) in edges and (x1+1,i) in edges:
            c+=1
            break
    for i in range(y2,yMax+1):
        if (x2,i) in edges and (x2+1,i) in edges:
            c+=1
            break
    if c==2: return True
    
    # checking for East and west
    c=0
    for i in range(x1+1,xMax+1):
        if (i,y1) in edges and (i,y2) in edges:
            c+=1
            break
    for i in range(xMin,x1):
        if (i,y1) in edges and (i,y2) in edges:
            c+=1
            break
    if c==2: return True
    return False    

In [84]:
isSegmentInPocket(edges,[(-5, 2), (-5, 3)],xyRanges)

False

In [41]:
ls=[[],
[],
[[(2, 2), (2, 3)], [(2, 3), (2, 4)]],
[[(3, 2), (3, 3)], [(3, 3), (3, 4)]],
[[(4, 2), (4, 3)], [(4, 3), (4, 4)], [(4, 4), (4, 5)], [(4, 5), (4, 6)]],
[[(5, 2), (5, 3)], [(5, 3), (5, 4)], [(5, 4), (5, 5)], [(5, 5), (5, 6)], [(5, 0), (5, 1)], [(5, 1), (5, 2)]]]

In [16]:
ls=[[],
[],
[],
[],
[[(-1, 0), (-1, 1)], [(-1, 1), (-1, 2)], [(-1, 2), (-1, 3)], [(-1, 3), (-1, 4)], [(-1, 4), (-1, 5)]],
[[(0, 0), (0, 1)], [(0, 1), (0, 2)], [(0, 2), (0, 3)], [(0, 3), (0, 4)], [(0, 4), (0, 5)], [(0, -3), (0, -2)], [(0, -2), (0, -1)], [(0, -1), (0, 0)]],
[[(1, 0), (1, 1)], [(1, 1), (1, 2)], [(1, 2), (1, 3)], [(1, 3), (1, 4)], [(1, 4), (1, 5)], [(1, -3), (1, -2)], [(1, -2), (1, -1)], [(1, -1), (1, 0)]],
[[(2, 0), (2, 1)], [(2, 1), (2, 2)], [(2, 2), (2, 3)], [(2, 3), (2, 4)], [(2, 4), (2, 5)], [(2, -3), (2, -2)], [(2, -2), (2, -1)], [(2, -1), (2, 0)]],
[],
[],
[],
[],
[[(7, -7), (7, -6)], [(7, -6), (7, -5)], [(7, -5), (7, -4)], [(7, -4), (7, -3)], [(7, -3), (7, -2)], [(7, -2), (7, -1)], [(7, -1), (7, 0)], [(7, 0), (7, 1)], [(7, 1), (7, 2)], [(7, 2), (7, 3)], [(7, 3), (7, 4)], [(7, 4), (7, 5)]]]

In [17]:
for i in ls:
    # if i:
    for j in i:
        print(isSegmentInPocket(edges,j,xyRanges),j)

True [(-1, 0), (-1, 1)]
True [(-1, 1), (-1, 2)]
True [(-1, 2), (-1, 3)]
True [(-1, 3), (-1, 4)]
True [(-1, 4), (-1, 5)]
True [(0, 0), (0, 1)]
True [(0, 1), (0, 2)]
True [(0, 2), (0, 3)]
True [(0, 3), (0, 4)]
True [(0, 4), (0, 5)]
True [(0, -3), (0, -2)]
True [(0, -2), (0, -1)]
True [(0, -1), (0, 0)]
True [(1, 0), (1, 1)]
True [(1, 1), (1, 2)]
True [(1, 2), (1, 3)]
True [(1, 3), (1, 4)]
True [(1, 4), (1, 5)]
True [(1, -3), (1, -2)]
True [(1, -2), (1, -1)]
True [(1, -1), (1, 0)]
True [(2, 0), (2, 1)]
True [(2, 1), (2, 2)]
True [(2, 2), (2, 3)]
True [(2, 3), (2, 4)]
True [(2, 4), (2, 5)]
True [(2, -3), (2, -2)]
True [(2, -2), (2, -1)]
True [(2, -1), (2, 0)]
False [(7, -7), (7, -6)]
False [(7, -6), (7, -5)]
False [(7, -5), (7, -4)]
False [(7, -4), (7, -3)]
False [(7, -3), (7, -2)]
False [(7, -2), (7, -1)]
False [(7, -1), (7, 0)]
False [(7, 0), (7, 1)]
False [(7, 1), (7, 2)]
False [(7, 2), (7, 3)]
False [(7, 3), (7, 4)]
False [(7, 4), (7, 5)]


In [15]:
isSegmentInPocket(edges,[(1,0),(1,-1)],xyRanges)

True

In [22]:
getArea(edges,xyRanges)

[[(-5, -7), (-5, -6)], [(-5, -6), (-5, -5)], [(-5, -5), (-5, -4)], [(-5, -4), (-5, -3)], [(-5, -3), (-5, -2)], [(-5, -2), (-5, -1)], [(-5, -1), (-5, 0)], [(-5, 0), (-5, 1)], [(-5, 1), (-5, 2)], [(-5, 2), (-5, 3)], [(-5, 3), (-5, 4)], [(-5, 4), (-5, 5)]]
[]
[]
[]
[]
[[(-1, 0), (-1, 1)], [(-1, 1), (-1, 2)], [(-1, 2), (-1, 3)], [(-1, 3), (-1, 4)], [(-1, 4), (-1, 5)]]
[[(0, 0), (0, 1)], [(0, 1), (0, 2)], [(0, 2), (0, 3)], [(0, 3), (0, 4)], [(0, 4), (0, 5)], [(0, -3), (0, -2)], [(0, -2), (0, -1)], [(0, -1), (0, 0)]]
[[(1, 0), (1, 1)], [(1, 1), (1, 2)], [(1, 2), (1, 3)], [(1, 3), (1, 4)], [(1, 4), (1, 5)], [(1, -3), (1, -2)], [(1, -2), (1, -1)], [(1, -1), (1, 0)]]
[[(2, 0), (2, 1)], [(2, 1), (2, 2)], [(2, 2), (2, 3)], [(2, 3), (2, 4)], [(2, 4), (2, 5)], [(2, -3), (2, -2)], [(2, -2), (2, -1)], [(2, -1), (2, 0)]]
[]
[]
[]
[]
[[(7, -7), (7, -6)], [(7, -6), (7, -5)], [(7, -5), (7, -4)], [(7, -4), (7, -3)], [(7, -3), (7, -2)], [(7, -2), (7, -1)], [(7, -1), (7, 0)], [(7, 0), (7, 1)], [(7, 1), (7, 

115

In [ ]:
for i in getSegments(edges,csa):
    print(i)

In [ ]:
a=[1,2,3,2,1]
a.remove(2)

In [ ]:
a=[{1,2},{3,4},{1,2}]

In [ ]:
a.remove({4,3})
a

In [ ]:
window=[[(5, 2),(5, 1)]]
xorSegments(window,[[(5, 0), (5, 1)],[(5, 1), (5, 2)]])

In [ ]:
window